In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding,LSTM,SpatialDropout1D
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils


In [2]:
Tweets=pd.read_csv('Tweets.csv')

In [4]:
Tweets.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)


In [8]:
Tweets.groupby(['airline_sentiment']).size()

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [9]:
Tweets = Tweets[Tweets['airline_sentiment_confidence']>0.8]

In [10]:
token = Tokenizer(num_words = 100)
token.fit_on_texts(Tweets['text'].values)

In [12]:
x=token.texts_to_sequences(Tweets['text'].values)
x = pad_sequences(x,padding='post',maxlen = 100)

In [13]:
print(x)

[[97 62  0 ...  0  0  0]
 [97 99  1 ...  0  0  0]
 [97  9 99 ...  0  0  0]
 ...
 [13 98 93 ...  0  0  0]
 [13 89  1 ...  0  0  0]
 [13  6 23 ...  0  0  0]]


In [14]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment'])
print(y)

[1 0 0 ... 0 1 0]


In [15]:
y = np_utils.to_categorical(y)
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [18]:
x_train,xtest,y_train,ytest = train_test_split(x, y, test_size=0.3,random_state=0)

In [19]:
xtest

array([[16, 14, 36, ...,  0,  0,  0],
       [13, 79, 38, ...,  0,  0,  0],
       [ 8, 10, 17, ...,  0,  0,  0],
       ...,
       [16, 14, 57, ...,  0,  0,  0],
       [16, 20, 92, ...,  0,  0,  0],
       [16, 80, 26, ...,  0,  0,  0]])

In [23]:
modelo = Sequential()

modelo.add(Embedding(input_dim=len(token.word_index),output_dim = 128,input_length= x.shape[1]))

modelo.add(SpatialDropout1D(0.2))
#units = quantidade de neuronio
modelo.add(LSTM(units = 196,dropout=0.2,recurrent_dropout=0,activation='tanh',recurrent_activation='sigmoid',unroll=False,use_bias=True))

modelo.add(Dense(units=3,activation='softmax'))

In [24]:
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1638656   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 1,894,047
Trainable params: 1,894,047
Non-trainable params: 0
_________________________________________________________________


In [26]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [27]:
modelo.fit(x_train,y_train,epochs=10,batch_size=30,verbose=True,validation_data=(xtest,ytest))

Epoch 1/10
245/245 [==============================] - 39s 152ms/step - loss: 0.8161 - accuracy: 0.7061 - val_loss: 0.8153 - val_accuracy: 0.7027
Epoch 2/10
245/245 [==============================] - 35s 145ms/step - loss: 0.8087 - accuracy: 0.7085 - val_loss: 0.8184 - val_accuracy: 0.7027
Epoch 3/10
245/245 [==============================] - 36s 146ms/step - loss: 0.8082 - accuracy: 0.7085 - val_loss: 0.8202 - val_accuracy: 0.7027
Epoch 4/10
245/245 [==============================] - 36s 146ms/step - loss: 0.8078 - accuracy: 0.7085 - val_loss: 0.8215 - val_accuracy: 0.7027
Epoch 5/10
245/245 [==============================] - 39s 160ms/step - loss: 0.8084 - accuracy: 0.7085 - val_loss: 0.8156 - val_accuracy: 0.7027
Epoch 6/10
245/245 [==============================] - 37s 150ms/step - loss: 0.8077 - accuracy: 0.7085 - val_loss: 0.8167 - val_accuracy: 0.7027
Epoch 7/10
245/245 [==============================] - 35s 143ms/step - loss: 0.8075 - accuracy: 0.7085 - val_loss: 0.8176 - val_ac

In [30]:
loss,accuracy = modelo.evaluate(xtest, ytest)
print("loss: ", loss,'\n','acuracia: ',accuracy)

99/99 [==============================] - 3s 27ms/step - loss: 0.8164 - accuracy: 0.7027
loss:  0.8164207935333252 
 acuracia:  0.7026768922805786


In [31]:
prev = modelo.predict(xtest)
prev

99/99 [==============================] - 3s 27ms/step


array([[0.7255598 , 0.13461064, 0.13982959],
       [0.7255597 , 0.13461064, 0.1398296 ],
       [0.7255598 , 0.13461064, 0.13982959],
       ...,
       [0.7255598 , 0.13461064, 0.13982959],
       [0.7255598 , 0.13461064, 0.13982959],
       [0.7255597 , 0.13461064, 0.1398296 ]], dtype=float32)